In [105]:
import torch
import scipy.io as io
import plotly.offline as py
import plotly.graph_objs as go
import json
import numpy as np
from sklearn.svm import SVC
from sklearn import metrics

import pickle 

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor

from sklearn.metrics import mean_squared_error
import math as m

py.init_notebook_mode(connected=True)

In [106]:
# Loading Matlab Struct array with all of the data:

mat = io.loadmat('data_Mg_GBperatom_seg_2Al_dump.mat')

length_A = mat['A'].shape[1]

In [107]:
# Organizing data:

for i in range(30):
    segE = mat['A']['Eseg'][0,i]
    #check whether this is a valid data?
    n1 = segE[:,0] != 0 
    segE = np.squeeze(segE[n1,:])
    atom_ID = segE[:,0].astype(int) - 1

    descriptor = mat['A']['peratom'][0,i][0,0]
    descriptor_temp = np.concatenate([descriptor['pos'],descriptor['pe'],descriptor['cna'],descriptor['centro_fnn'],
                                descriptor['centro_snn'],descriptor['coord'],descriptor['f'],descriptor['stress'],
                                descriptor['voronoi']], axis = 1)
    if i == 0:
        descriptor_all = descriptor_temp[atom_ID]
        segE_all = segE
    else:
        descriptor_temp = descriptor_temp[atom_ID]
        descriptor_all = np.concatenate([descriptor_all, descriptor_temp], axis = 0)
        segE_all = np.concatenate([segE_all, segE])

descriptor_all[:,2] = abs(descriptor_all[:,2]-min(descriptor_all[:,2])-20)
sigma_H = np.sum(descriptor_all[:,11:14], axis = 1)/3
f_mag = np.linalg.norm(descriptor_all[:,8:11], axis = 1, ord = 2)

feature = np.concatenate([descriptor_all, sigma_H[:,np.newaxis], f_mag[:,np.newaxis]], axis = 1)

# Quadratic Regression with Interaction Terms

In [135]:
y_true = segE_all[:,1]
feature1 = feature[:,3:]
pos = feature[:,:3]
zpos = pos[:,2][:,np.newaxis]
ypos= pos[:,1][:,np.newaxis]
xpos= pos[:,0][:,np.newaxis]

feature1 = feature[:,3:]

# Separation of each feature (descriptor) other than position, which was done above:
f0= feature1[:,0][:,np.newaxis]
f1= feature1[:,1][:,np.newaxis]
f2= feature1[:,2][:,np.newaxis]
f3= feature1[:,3][:,np.newaxis]
f4= feature1[:,4][:,np.newaxis]
f5= feature1[:,5][:,np.newaxis]
f6= feature1[:,6][:,np.newaxis]
f7= feature1[:,7][:,np.newaxis]
f8= feature1[:,8][:,np.newaxis]
f9= feature1[:,9][:,np.newaxis]
f10= feature1[:,10][:,np.newaxis]
f11= feature1[:,11][:,np.newaxis]
f12= feature1[:,12][:,np.newaxis]
f13= feature1[:,13][:,np.newaxis]
f14= feature1[:,14][:,np.newaxis]
f15= feature1[:,15][:,np.newaxis]
f16= feature1[:,16][:,np.newaxis]
f17= feature1[:,17][:,np.newaxis]

# Feature Reduction (as determined by Recursive Feature Elimination strategy- See bottom of 'Base Models' file for example)
feature1 = np.concatenate([f0,f1,f2,f3,f4,f8,f9,f10,f11,f13,f14,f15,f16,xpos,ypos], axis=1)
print(feature1.shape)

n= feature1.shape[1]
print(n)
feature1 = feature1/np.max(feature1)
feature2 = np.zeros([feature1.shape[0], n*n])
print(feature2.shape)
# print(feature2)
for i in range(n):
    feature2[:,i*n:i*n+n] = feature1[:,i][:,np.newaxis]*feature1[:,:]

feature_space = np.concatenate([feature1, feature2], axis = 1)
print(feature_space.shape)
feature_space = np.concatenate([np.ones((feature1.shape[0],1)), feature_space], axis = 1)
print(feature_space.shape)




np.random.seed(10)
idx0 = np.random.permutation(np.arange(len(feature_space)))
feature_space = feature_space[idx0]
y_true = y_true[idx0]




X_fold1 = feature_space[0:int(len(feature_space)*.2)]
X_fold2 = feature_space[int(len(feature_space)*.2):int(len(feature_space)*.4)]
X_fold3 = feature_space[int(len(feature_space)*.4):int(len(feature_space)*.6)]
X_fold4 = feature_space[int(len(feature_space)*.6):int(len(feature_space)*.8)]
X_fold5 = feature_space[int(len(feature_space)*.8):]

X_std1 = np.std(X_fold1, axis = 0)
X_std2 = np.std(X_fold2, axis = 0)
X_std3 = np.std(X_fold3, axis = 0)
X_std4 = np.std(X_fold4, axis = 0)
X_std5 = np.std(X_fold5, axis = 0)

X_mean1 = np.mean(X_fold1, axis = 0)
X_mean2 = np.mean(X_fold2, axis = 0)
X_mean3 = np.mean(X_fold3, axis = 0)
X_mean4 = np.mean(X_fold4, axis = 0)
X_mean5 = np.mean(X_fold5, axis = 0)

# X_fold1 = (X_fold1 - X_mean1)/X_std1         #No normalization for our linear models
# X_fold2 = (X_fold2 - X_mean2)/X_std2
# X_fold3 = (X_fold3 - X_mean3)/X_std3
# X_fold4 = (X_fold4 - X_mean4)/X_std4
# X_fold5 = (X_fold5 - X_mean5)/X_std5


y_fold1 = y_true[0:int(len(feature_space)*.2)]
y_fold2 = y_true[int(len(feature_space)*.2):int(len(feature_space)*.4)]
y_fold3 = y_true[int(len(feature_space)*.4):int(len(feature_space)*.6)]
y_fold4 = y_true[int(len(feature_space)*.6):int(len(feature_space)*.8)]
y_fold5 = y_true[int(len(feature_space)*.8):]



print(X_fold1.shape)
print(X_fold2.shape)
print(X_fold3.shape)
print(X_fold4.shape)
print(X_fold5.shape,'\n')

print(y_fold1.shape)
print(y_fold2.shape)
print(y_fold3.shape)
print(y_fold4.shape)
print(y_fold5.shape, '\n')



X_train1 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold4], axis=0)
X_test1 = X_fold5
y_train1 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold4], axis=0)
y_test1 = y_fold5

X_train2 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold5], axis=0)
X_test2= X_fold4
y_train2 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold5], axis=0)
y_test2 = y_fold4

X_train3 = np.concatenate([X_fold1,X_fold2,X_fold4,X_fold5], axis=0)
X_test3 = X_fold3
y_train3 = np.concatenate([y_fold1,y_fold2,y_fold4,y_fold5], axis=0)
y_test3 = y_fold3

X_train4 = np.concatenate([X_fold1,X_fold3,X_fold4,X_fold5], axis=0)
X_test4 = X_fold2
y_train4 = np.concatenate([y_fold1,y_fold3,y_fold4,y_fold5], axis=0)
y_test4 = y_fold2

X_train5 = np.concatenate([X_fold2,X_fold3,X_fold4,X_fold5], axis=0)
X_test5 = X_fold1
y_train5 = np.concatenate([y_fold2,y_fold3,y_fold4,y_fold5], axis=0)
y_test5 = y_fold1


(14231, 15)
15
(14231, 225)
(14231, 240)
(14231, 241)
(2846, 241)
(2846, 241)
(2846, 241)
(2846, 241)
(2847, 241) 

(2846,)
(2846,)
(2846,)
(2846,)
(2847,) 



In [136]:

R2_train_quad_int_list = []
R2_val_quad_int_list = []
rmse_quad_int_list = []

In [137]:
# w_pre1=np.linalg.pinv(X_train1).dot(y_train1)  #
# y1_pred1 = X_train1.dot(w_pre1)                 #
# np.power(np.linalg.norm(y1_pred1-y_train1), 2)  #


# Y_pred_train1 = X_train1.dot(w_pre1)        #
# Y_mean = np.mean(y_train1)                  #
# SS_tot = np.sum(np.power(y_train1 - Y_mean, 2))   #
# SS_res = np.power(np.linalg.norm(Y_pred_train1-y_train1), 2)   #
# R_squared_train = 1-SS_res/SS_tot
# print('R^2 train:', R_squared_train)   


# Y_pred_test1 = X_test1.dot(w_pre1)     #
# Y_mean = np.mean(y_test1)    #
# SS_tot = np.sum(np.power(y_test1 - Y_mean, 2))   #
# SS_res = np.power(np.linalg.norm(Y_pred_test1-y_test1), 2)  #
# R_squared_val = 1-SS_res/SS_tot
# print('R^2 val:', R_squared_val)
# print('\n')



X_train11 = X_train1
y_train11 = y_train1
X_test11 = X_test1
y_test11 = y_test1

w_pre11=np.linalg.pinv(X_train11).dot(y_train11)  #
y1_pred11 = X_train11.dot(w_pre11)                 #
np.power(np.linalg.norm(y1_pred11-y_train11), 2)  #


Y_pred_train11 = X_train11.dot(w_pre11)        #
Y_mean = np.mean(y_train11)                  #
SS_tot = np.sum(np.power(y_train11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_train11-y_train11), 2)   #
R_squared_train = 1-SS_res/SS_tot
print('R^2 train:', R_squared_train)   


Y_pred_test11 = X_test11.dot(w_pre11)     #
Y_mean = np.mean(y_test11)    #
SS_tot = np.sum(np.power(y_test11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_test11-y_test11), 2)  #
R_squared_val = 1-SS_res/SS_tot
print('R^2 val:', R_squared_val)

rmse = np.sqrt(mean_squared_error(y_test11,Y_pred_test11))
print('RMSE:',rmse)
rmse_quad_int_list.append(rmse)


R2_train_quad_int_list.append(R_squared_train)
R2_val_quad_int_list.append(R_squared_val)


R^2 train: 0.9409657672871464
R^2 val: 0.9409483329179229
RMSE: 0.011598069885265608


In [138]:
X_train11 = X_train2
y_train11 = y_train2
X_test11 = X_test2
y_test11 = y_test2

w_pre11=np.linalg.pinv(X_train11).dot(y_train11)  #
y1_pred11 = X_train11.dot(w_pre11)                 #
# np.power(np.linalg.norm(y1_pred11-y_train11), 2)  #


Y_pred_train11 = X_train11.dot(w_pre11)        #
Y_mean = np.mean(y_train11)                  #
SS_tot = np.sum(np.power(y_train11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_train11-y_train11), 2)   #
R_squared_train = 1-SS_res/SS_tot
print('R^2 train:', R_squared_train)   


Y_pred_test11 = X_test11.dot(w_pre11)     #
Y_mean = np.mean(y_test11)    #
SS_tot = np.sum(np.power(y_test11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_test11-y_test11), 2)  #
R_squared_val = 1-SS_res/SS_tot
print('R^2 val:', R_squared_val)


rmse = np.sqrt(mean_squared_error(y_test11,Y_pred_test11))
print('RMSE:',rmse)
rmse_quad_int_list.append(rmse)

R2_train_quad_int_list.append(R_squared_train)
R2_val_quad_int_list.append(R_squared_val)

R^2 train: 0.9421454403300437
R^2 val: 0.9363578904906841
RMSE: 0.011247956161937984


In [139]:
X_train11 = X_train3
y_train11 = y_train3
X_test11 = X_test3
y_test11 = y_test3

w_pre11=np.linalg.pinv(X_train11).dot(y_train11)  #
y1_pred11 = X_train11.dot(w_pre11)                 #
# np.power(np.linalg.norm(y1_pred11-y_train11), 2)  #


Y_pred_train11 = X_train11.dot(w_pre11)        #
Y_mean = np.mean(y_train11)                  #
SS_tot = np.sum(np.power(y_train11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_train11-y_train11), 2)   #
R_squared_train = 1-SS_res/SS_tot
print('R^2 train:', R_squared_train)   


Y_pred_test11 = X_test11.dot(w_pre11)     #
Y_mean = np.mean(y_test11)    #
SS_tot = np.sum(np.power(y_test11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_test11-y_test11), 2)  #
R_squared_val = 1-SS_res/SS_tot
print('R^2 val:', R_squared_val)


rmse = np.sqrt(mean_squared_error(y_test11,Y_pred_test11))
print('RMSE:',rmse)
rmse_quad_int_list.append(rmse)

R2_train_quad_int_list.append(R_squared_train)
R2_val_quad_int_list.append(R_squared_val)

R^2 train: 0.9427618743769485
R^2 val: 0.9344387916655394
RMSE: 0.011467213969854902


In [140]:
X_train11 = X_train4
y_train11 = y_train4
X_test11 = X_test4
y_test11 = y_test4

w_pre11=np.linalg.pinv(X_train11).dot(y_train11)  #
y1_pred11 = X_train11.dot(w_pre11)                 #
# np.power(np.linalg.norm(y1_pred11-y_train11), 2)  #


Y_pred_train11 = X_train11.dot(w_pre11)        #
Y_mean = np.mean(y_train11)                  #
SS_tot = np.sum(np.power(y_train11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_train11-y_train11), 2)   #
R_squared_train = 1-SS_res/SS_tot
print('R^2 train:', R_squared_train)   


Y_pred_test11 = X_test11.dot(w_pre11)     #
Y_mean = np.mean(y_test11)    #
SS_tot = np.sum(np.power(y_test11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_test11-y_test11), 2)  #
R_squared_val = 1-SS_res/SS_tot
print('R^2 val:', R_squared_val)


rmse = np.sqrt(mean_squared_error(y_test11,Y_pred_test11))
print('RMSE:',rmse)
rmse_quad_int_list.append(rmse)




R2_train_quad_int_list.append(R_squared_train)
R2_val_quad_int_list.append(R_squared_val)

R^2 train: 0.9430556140027982
R^2 val: 0.9280043731863215
RMSE: 0.011832656363787816


In [141]:
X_train11 = X_train5
y_train11 = y_train5
X_test11 = X_test5
y_test11 = y_test5

w_pre11=np.linalg.pinv(X_train11).dot(y_train11)  #
y1_pred11 = X_train11.dot(w_pre11)                 #
# np.power(np.linalg.norm(y1_pred11-y_train11), 2)  #


Y_pred_train11 = X_train11.dot(w_pre11)        #
Y_mean = np.mean(y_train11)                  #
SS_tot = np.sum(np.power(y_train11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_train11-y_train11), 2)   #
R_squared_train = 1-SS_res/SS_tot
print('R^2 train:', R_squared_train)   


Y_pred_test11 = X_test11.dot(w_pre11)     #
Y_mean = np.mean(y_test11)    #
SS_tot = np.sum(np.power(y_test11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_test11-y_test11), 2)  #
R_squared_val = 1-SS_res/SS_tot
print('R^2 val:', R_squared_val)


rmse = np.sqrt(mean_squared_error(y_test11,Y_pred_test11))
print('RMSE:',rmse)
rmse_quad_int_list.append(rmse)



R2_train_quad_int_list.append(R_squared_train)
R2_val_quad_int_list.append(R_squared_val)

R^2 train: 0.9412548582956379
R^2 val: 0.9410390292386404
RMSE: 0.011200628773322974


In [142]:
print(R2_train_quad_int_list)
print(R2_val_quad_int_list)
print(rmse_quad_int_list)

[0.9409657672871464, 0.9421454403300437, 0.9427618743769485, 0.9430556140027982, 0.9412548582956379]
[0.9409483329179229, 0.9363578904906841, 0.9344387916655394, 0.9280043731863215, 0.9410390292386404]
[0.011598069885265608, 0.011247956161937984, 0.011467213969854902, 0.011832656363787816, 0.011200628773322974]


In [143]:
print(np.std(R2_train_quad_int_list))
print(np.std(R2_val_quad_int_list))
print(np.std(rmse_quad_int_list))

0.0008165857465091001
0.004821804623051796
0.00023228913213656964


In [ ]:
[0.9409657672871464, 0.9421454403300437, 0.9427618743769485, 0.9430556140027982, 0.9412548582956379]
[0.9409483329179229, 0.9363578904906841, 0.9344387916655394, 0.9280043731863215, 0.9410390292386404]
[0.013821271670408617, 0.01331369783271392, 0.014345471054398748, 0.013426798682177664, 0.013683884292652567]

# Quadratic Regression w/o Interaction Terms

In [144]:
y_true = segE_all[:,1]
pos = feature[:,:3]
zpos = pos[:,2][:,np.newaxis]
ypos= pos[:,1][:,np.newaxis]
xpos= pos[:,0][:,np.newaxis]

feature1 = feature[:,3:]
# Separation of each feature:
f0= feature1[:,0][:,np.newaxis]
f1= feature1[:,1][:,np.newaxis]
f2= feature1[:,2][:,np.newaxis]
f3= feature1[:,3][:,np.newaxis]
f4= feature1[:,4][:,np.newaxis]
f5= feature1[:,5][:,np.newaxis]
f6= feature1[:,6][:,np.newaxis]
f7= feature1[:,7][:,np.newaxis]
f8= feature1[:,8][:,np.newaxis]
f9= feature1[:,9][:,np.newaxis]
f10= feature1[:,10][:,np.newaxis]
f11= feature1[:,11][:,np.newaxis]
f12= feature1[:,12][:,np.newaxis]
f13= feature1[:,13][:,np.newaxis]
f14= feature1[:,14][:,np.newaxis]
f15= feature1[:,15][:,np.newaxis]
f16= feature1[:,16][:,np.newaxis]
f17= feature1[:,17][:,np.newaxis]

feature1 = np.concatenate([f0,f1,f2,f3,f4,f8,f9,f10,f11,f13,f14,f15,f16,xpos,ypos], axis=1)
print(feature1.shape)

y_true = segE_all[:,1]

feature1 = feature1/np.max(feature1)
feature2 = np.power(feature1, 2)
print(feature2.shape)
feature_space = np.concatenate([feature1, feature2], axis = 1)
print(feature_space.shape)
feature_space = np.concatenate([np.ones((feature1.shape[0],1)), feature_space], axis = 1)
print(feature_space.shape)
np.random.seed(10)
idx0 = np.random.permutation(np.arange(len(feature_space)))
feature_space = feature_space[idx0]
y_true = y_true[idx0]


X_fold1 = feature_space[0:int(len(feature_space)*.2)]
X_fold2 = feature_space[int(len(feature_space)*.2):int(len(feature_space)*.4)]
X_fold3 = feature_space[int(len(feature_space)*.4):int(len(feature_space)*.6)]
X_fold4 = feature_space[int(len(feature_space)*.6):int(len(feature_space)*.8)]
X_fold5 = feature_space[int(len(feature_space)*.8):]

X_std1 = np.std(X_fold1, axis = 0)
X_std2 = np.std(X_fold2, axis = 0)
X_std3 = np.std(X_fold3, axis = 0)
X_std4 = np.std(X_fold4, axis = 0)
X_std5 = np.std(X_fold5, axis = 0)

X_mean1 = np.mean(X_fold1, axis = 0)
X_mean2 = np.mean(X_fold2, axis = 0)
X_mean3 = np.mean(X_fold3, axis = 0)
X_mean4 = np.mean(X_fold4, axis = 0)
X_mean5 = np.mean(X_fold5, axis = 0)

# X_fold1 = (X_fold1 - X_mean1)/X_std1         #No normalization for our linear models
# X_fold2 = (X_fold2 - X_mean2)/X_std2
# X_fold3 = (X_fold3 - X_mean3)/X_std3
# X_fold4 = (X_fold4 - X_mean4)/X_std4
# X_fold5 = (X_fold5 - X_mean5)/X_std5


y_fold1 = y_true[0:int(len(feature_space)*.2)]
y_fold2 = y_true[int(len(feature_space)*.2):int(len(feature_space)*.4)]
y_fold3 = y_true[int(len(feature_space)*.4):int(len(feature_space)*.6)]
y_fold4 = y_true[int(len(feature_space)*.6):int(len(feature_space)*.8)]
y_fold5 = y_true[int(len(feature_space)*.8):]



print(X_fold1.shape)
print(X_fold2.shape)
print(X_fold3.shape)
print(X_fold4.shape)
print(X_fold5.shape,'\n')

print(y_fold1.shape)
print(y_fold2.shape)
print(y_fold3.shape)
print(y_fold4.shape)
print(y_fold5.shape, '\n')



X_train1 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold4], axis=0)
X_test1 = X_fold5
y_train1 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold4], axis=0)
y_test1 = y_fold5

X_train2 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold5], axis=0)
X_test2= X_fold4
y_train2 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold5], axis=0)
y_test2 = y_fold4

X_train3 = np.concatenate([X_fold1,X_fold2,X_fold4,X_fold5], axis=0)
X_test3 = X_fold3
y_train3 = np.concatenate([y_fold1,y_fold2,y_fold4,y_fold5], axis=0)
y_test3 = y_fold3

X_train4 = np.concatenate([X_fold1,X_fold3,X_fold4,X_fold5], axis=0)
X_test4 = X_fold2
y_train4 = np.concatenate([y_fold1,y_fold3,y_fold4,y_fold5], axis=0)
y_test4 = y_fold2

X_train5 = np.concatenate([X_fold2,X_fold3,X_fold4,X_fold5], axis=0)
X_test5 = X_fold1
y_train5 = np.concatenate([y_fold2,y_fold3,y_fold4,y_fold5], axis=0)
y_test5 = y_fold1


(14231, 15)
(14231, 15)
(14231, 30)
(14231, 31)
(2846, 31)
(2846, 31)
(2846, 31)
(2846, 31)
(2847, 31) 

(2846,)
(2846,)
(2846,)
(2846,)
(2847,) 



In [145]:
R2_train_quad_noint_list = []
R2_val_quad_noint_list = []
rmse_quad_noint_list = []

In [146]:
X_train11 = X_train1
y_train11 = y_train1
X_test11 = X_test1
y_test11 = y_test1

w_pre11=np.linalg.pinv(X_train11).dot(y_train11)  #
y1_pred11 = X_train11.dot(w_pre11)                 #
np.power(np.linalg.norm(y1_pred11-y_train11), 2)  #


Y_pred_train11 = X_train11.dot(w_pre11)        #
Y_mean = np.mean(y_train11)                  #
SS_tot = np.sum(np.power(y_train11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_train11-y_train11), 2)   #
R_squared_train = 1-SS_res/SS_tot
print('R^2 train:', R_squared_train)   


Y_pred_test11 = X_test11.dot(w_pre11)     #
Y_mean = np.mean(y_test11)    #
SS_tot = np.sum(np.power(y_test11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_test11-y_test11), 2)  #
R_squared_val = 1-SS_res/SS_tot
print('R^2 val:', R_squared_val)


rmse = np.sqrt(mean_squared_error(y_test11,Y_pred_test11))
print('RMSE:',rmse)
rmse_quad_noint_list.append(rmse)


R2_train_quad_noint_list.append(R_squared_train)
R2_val_quad_noint_list.append(R_squared_val)


R^2 train: 0.9082537685175032
R^2 val: 0.9161396390969365
RMSE: 0.013821271670408617


In [147]:
X_train11 = X_train2
y_train11 = y_train2
X_test11 = X_test2
y_test11 = y_test2

w_pre11=np.linalg.pinv(X_train11).dot(y_train11)  #
y1_pred11 = X_train11.dot(w_pre11)                 #
np.power(np.linalg.norm(y1_pred11-y_train11), 2)  #


Y_pred_train11 = X_train11.dot(w_pre11)        #
Y_mean = np.mean(y_train11)                  #
SS_tot = np.sum(np.power(y_train11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_train11-y_train11), 2)   #
R_squared_train = 1-SS_res/SS_tot
print('R^2 train:', R_squared_train)   


Y_pred_test11 = X_test11.dot(w_pre11)     #
Y_mean = np.mean(y_test11)    #
SS_tot = np.sum(np.power(y_test11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_test11-y_test11), 2)  #
R_squared_val = 1-SS_res/SS_tot
print('R^2 val:', R_squared_val)


rmse = np.sqrt(mean_squared_error(y_test11,Y_pred_test11))
print('RMSE:',rmse)
rmse_quad_noint_list.append(rmse)

R2_train_quad_noint_list.append(R_squared_train)
R2_val_quad_noint_list.append(R_squared_val)


R^2 train: 0.9097606744925801
R^2 val: 0.9108349354281575
RMSE: 0.01331369783271392


In [148]:
X_train11 = X_train3
y_train11 = y_train3
X_test11 = X_test3
y_test11 = y_test3

w_pre11=np.linalg.pinv(X_train11).dot(y_train11)  #
y1_pred11 = X_train11.dot(w_pre11)                 #
np.power(np.linalg.norm(y1_pred11-y_train11), 2)  #


Y_pred_train11 = X_train11.dot(w_pre11)        #
Y_mean = np.mean(y_train11)                  #
SS_tot = np.sum(np.power(y_train11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_train11-y_train11), 2)   #
R_squared_train = 1-SS_res/SS_tot
print('R^2 train:', R_squared_train)   


Y_pred_test11 = X_test11.dot(w_pre11)     #
Y_mean = np.mean(y_test11)    #
SS_tot = np.sum(np.power(y_test11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_test11-y_test11), 2)  #
R_squared_val = 1-SS_res/SS_tot
print('R^2 val:', R_squared_val)


rmse = np.sqrt(mean_squared_error(y_test11,Y_pred_test11))
print('RMSE:',rmse)
rmse_quad_noint_list.append(rmse)

R2_train_quad_noint_list.append(R_squared_train)
R2_val_quad_noint_list.append(R_squared_val)


R^2 train: 0.9130309911116625
R^2 val: 0.8973968382512306
RMSE: 0.014345471054398748


In [149]:
X_train11 = X_train4
y_train11 = y_train4
X_test11 = X_test4
y_test11 = y_test4

w_pre11=np.linalg.pinv(X_train11).dot(y_train11)  #
y1_pred11 = X_train11.dot(w_pre11)                 #
np.power(np.linalg.norm(y1_pred11-y_train11), 2)  #


Y_pred_train11 = X_train11.dot(w_pre11)        #
Y_mean = np.mean(y_train11)                  #
SS_tot = np.sum(np.power(y_train11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_train11-y_train11), 2)   #
R_squared_train = 1-SS_res/SS_tot
print('R^2 train:', R_squared_train)   


Y_pred_test11 = X_test11.dot(w_pre11)     #
Y_mean = np.mean(y_test11)    #
SS_tot = np.sum(np.power(y_test11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_test11-y_test11), 2)  #
R_squared_val = 1-SS_res/SS_tot
print('R^2 val:', R_squared_val)


rmse = np.sqrt(mean_squared_error(y_test11,Y_pred_test11))
print('RMSE:',rmse)
rmse_quad_noint_list.append(rmse)

R2_train_quad_noint_list.append(R_squared_train)
R2_val_quad_noint_list.append(R_squared_val)


R^2 train: 0.9105168461882807
R^2 val: 0.9072985414581426
RMSE: 0.013426798682177664


In [150]:
X_train11 = X_train5
y_train11 = y_train5
X_test11 = X_test5
y_test11 = y_test5

w_pre11=np.linalg.pinv(X_train11).dot(y_train11)  #
y1_pred11 = X_train11.dot(w_pre11)                 #
np.power(np.linalg.norm(y1_pred11-y_train11), 2)  #


Y_pred_train11 = X_train11.dot(w_pre11)        #
Y_mean = np.mean(y_train11)                  #
SS_tot = np.sum(np.power(y_train11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_train11-y_train11), 2)   #
R_squared_train = 1-SS_res/SS_tot
print('R^2 train:', R_squared_train)   


Y_pred_test11 = X_test11.dot(w_pre11)     #
Y_mean = np.mean(y_test11)    #
SS_tot = np.sum(np.power(y_test11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_test11-y_test11), 2)  #
R_squared_val = 1-SS_res/SS_tot
print('R^2 val:', R_squared_val)


rmse = np.sqrt(mean_squared_error(y_test11,Y_pred_test11))
print('RMSE:',rmse)
rmse_quad_noint_list.append(rmse)

R2_train_quad_noint_list.append(R_squared_train)
R2_val_quad_noint_list.append(R_squared_val)


R^2 train: 0.909470334791712
R^2 val: 0.911996771534374
RMSE: 0.013683884292652567


In [153]:
print(R2_train_quad_noint_list)
print(R2_val_quad_noint_list)
print(rmse_quad_noint_list)

[0.9082537685175032, 0.9097606744925801, 0.9130309911116625, 0.9105168461882807, 0.909470334791712]
[0.9161396390969365, 0.9108349354281575, 0.8973968382512306, 0.9072985414581426, 0.911996771534374]
[0.013821271670408617, 0.01331369783271392, 0.014345471054398748, 0.013426798682177664, 0.013683884292652567]


In [154]:
print(np.std(R2_train_quad_noint_list))
print(np.std(R2_val_quad_noint_list))
print(np.std(rmse_quad_noint_list))

0.0015892088996406614
0.006332375127737084
0.00036160946918292286


# First Order Regression

In [124]:
y_true = segE_all[:,1]

pos = feature[:,:3]
zpos = pos[:,2][:,np.newaxis]
ypos= pos[:,1][:,np.newaxis]
xpos= pos[:,0][:,np.newaxis]

feature1 = feature[:,3:]
# Separation of each feature:
f0= feature1[:,0][:,np.newaxis]
f1= feature1[:,1][:,np.newaxis]
f2= feature1[:,2][:,np.newaxis]
f3= feature1[:,3][:,np.newaxis]
f4= feature1[:,4][:,np.newaxis]
f5= feature1[:,5][:,np.newaxis]
f6= feature1[:,6][:,np.newaxis]
f7= feature1[:,7][:,np.newaxis]
f8= feature1[:,8][:,np.newaxis]
f9= feature1[:,9][:,np.newaxis]
f10= feature1[:,10][:,np.newaxis]
f11= feature1[:,11][:,np.newaxis]
f12= feature1[:,12][:,np.newaxis]
f13= feature1[:,13][:,np.newaxis]
f14= feature1[:,14][:,np.newaxis]
f15= feature1[:,15][:,np.newaxis]
f16= feature1[:,16][:,np.newaxis]
f17= feature1[:,17][:,np.newaxis]

# Feature Reduction (as determined by Recursive Feature Elimination strategy)
feature1 = np.concatenate([f0,f1,f2,f3,f4,f8,f9,f10,f12,f13,f14,f15,f16,xpos,ypos], axis=1)
print(feature1.shape)

feature1 = feature1/np.max(feature1)
feature_space = np.concatenate([np.ones((feature1.shape[0],1)), feature1], axis = 1)
w = np.random.randn(38)
λ = 0

#random shuffle
idx0 = np.random.permutation(np.arange(len(feature_space)))
feature_space = feature_space[idx0]
y_true = y_true[idx0]






X_fold1 = feature_space[0:int(len(feature_space)*.2)]
X_fold2 = feature_space[int(len(feature_space)*.2):int(len(feature_space)*.4)]
X_fold3 = feature_space[int(len(feature_space)*.4):int(len(feature_space)*.6)]
X_fold4 = feature_space[int(len(feature_space)*.6):int(len(feature_space)*.8)]
X_fold5 = feature_space[int(len(feature_space)*.8):]

X_std1 = np.std(X_fold1, axis = 0)
X_std2 = np.std(X_fold2, axis = 0)
X_std3 = np.std(X_fold3, axis = 0)
X_std4 = np.std(X_fold4, axis = 0)
X_std5 = np.std(X_fold5, axis = 0)

X_mean1 = np.mean(X_fold1, axis = 0)
X_mean2 = np.mean(X_fold2, axis = 0)
X_mean3 = np.mean(X_fold3, axis = 0)
X_mean4 = np.mean(X_fold4, axis = 0)
X_mean5 = np.mean(X_fold5, axis = 0)

# X_fold1 = (X_fold1 - X_mean1)/X_std1         #No normalization for our linear models
# X_fold2 = (X_fold2 - X_mean2)/X_std2
# X_fold3 = (X_fold3 - X_mean3)/X_std3
# X_fold4 = (X_fold4 - X_mean4)/X_std4
# X_fold5 = (X_fold5 - X_mean5)/X_std5


y_fold1 = y_true[0:int(len(feature_space)*.2)]
y_fold2 = y_true[int(len(feature_space)*.2):int(len(feature_space)*.4)]
y_fold3 = y_true[int(len(feature_space)*.4):int(len(feature_space)*.6)]
y_fold4 = y_true[int(len(feature_space)*.6):int(len(feature_space)*.8)]
y_fold5 = y_true[int(len(feature_space)*.8):]



print(X_fold1.shape)
print(X_fold2.shape)
print(X_fold3.shape)
print(X_fold4.shape)
print(X_fold5.shape,'\n')

print(y_fold1.shape)
print(y_fold2.shape)
print(y_fold3.shape)
print(y_fold4.shape)
print(y_fold5.shape, '\n')



X_train1 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold4], axis=0)
X_test1 = X_fold5
y_train1 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold4], axis=0)
y_test1 = y_fold5

X_train2 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold5], axis=0)
X_test2= X_fold4
y_train2 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold5], axis=0)
y_test2 = y_fold4

X_train3 = np.concatenate([X_fold1,X_fold2,X_fold4,X_fold5], axis=0)
X_test3 = X_fold3
y_train3 = np.concatenate([y_fold1,y_fold2,y_fold4,y_fold5], axis=0)
y_test3 = y_fold3

X_train4 = np.concatenate([X_fold1,X_fold3,X_fold4,X_fold5], axis=0)
X_test4 = X_fold2
y_train4 = np.concatenate([y_fold1,y_fold3,y_fold4,y_fold5], axis=0)
y_test4 = y_fold2

X_train5 = np.concatenate([X_fold2,X_fold3,X_fold4,X_fold5], axis=0)
X_test5 = X_fold1
y_train5 = np.concatenate([y_fold2,y_fold3,y_fold4,y_fold5], axis=0)
y_test5 = y_fold1


(14231, 15)
(2846, 16)
(2846, 16)
(2846, 16)
(2846, 16)
(2847, 16) 

(2846,)
(2846,)
(2846,)
(2846,)
(2847,) 



In [125]:
R2_train_lin_list = []
R2_val_lin_list = []
rmse_lin_list = []

In [126]:
X_train11 = X_train1
y_train11 = y_train1
X_test11 = X_test1
y_test11 = y_test1

w_pre11=np.linalg.pinv(X_train11).dot(y_train11)  #
y1_pred11 = X_train11.dot(w_pre11)                 #
np.power(np.linalg.norm(y1_pred11-y_train11), 2)  #


Y_pred_train11 = X_train11.dot(w_pre11)        #
Y_mean = np.mean(y_train11)                  #
SS_tot = np.sum(np.power(y_train11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_train11-y_train11), 2)   #
R_squared_train = 1-SS_res/SS_tot
print('R^2 train:', R_squared_train)   


Y_pred_test11 = X_test11.dot(w_pre11)     #
Y_mean = np.mean(y_test11)    #
SS_tot = np.sum(np.power(y_test11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_test11-y_test11), 2)  #
R_squared_val = 1-SS_res/SS_tot
print('R^2 val:', R_squared_val)


rmse = np.sqrt(mean_squared_error(y_test11,Y_pred_test11))
print('RMSE:',rmse)
rmse_lin_list.append(rmse)

R2_train_lin_list.append(R_squared_train)
R2_val_lin_list.append(R_squared_val)


R^2 train: 0.8792878012535912
R^2 val: 0.8847739876333096
RMSE: 0.015661128397954457


In [127]:
X_train11 = X_train2
y_train11 = y_train2
X_test11 = X_test2
y_test11 = y_test2

w_pre11=np.linalg.pinv(X_train11).dot(y_train11)  #
y1_pred11 = X_train11.dot(w_pre11)                 #
np.power(np.linalg.norm(y1_pred11-y_train11), 2)  #


Y_pred_train11 = X_train11.dot(w_pre11)        #
Y_mean = np.mean(y_train11)                  #
SS_tot = np.sum(np.power(y_train11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_train11-y_train11), 2)   #
R_squared_train = 1-SS_res/SS_tot
print('R^2 train:', R_squared_train)   


Y_pred_test11 = X_test11.dot(w_pre11)     #
Y_mean = np.mean(y_test11)    #
SS_tot = np.sum(np.power(y_test11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_test11-y_test11), 2)  #
R_squared_val = 1-SS_res/SS_tot
print('R^2 val:', R_squared_val)


rmse = np.sqrt(mean_squared_error(y_test11,Y_pred_test11))
print('RMSE:',rmse)
rmse_lin_list.append(rmse)

R2_train_lin_list.append(R_squared_train)
R2_val_lin_list.append(R_squared_val)


R^2 train: 0.8822669543759765
R^2 val: 0.8723043781349946
RMSE: 0.01619188284715462


In [128]:
X_train11 = X_train3
y_train11 = y_train3
X_test11 = X_test3
y_test11 = y_test3

w_pre11=np.linalg.pinv(X_train11).dot(y_train11)  #
y1_pred11 = X_train11.dot(w_pre11)                 #
np.power(np.linalg.norm(y1_pred11-y_train11), 2)  #


Y_pred_train11 = X_train11.dot(w_pre11)        #
Y_mean = np.mean(y_train11)                  #
SS_tot = np.sum(np.power(y_train11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_train11-y_train11), 2)   #
R_squared_train = 1-SS_res/SS_tot
print('R^2 train:', R_squared_train)   


Y_pred_test11 = X_test11.dot(w_pre11)     #
Y_mean = np.mean(y_test11)    #
SS_tot = np.sum(np.power(y_test11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_test11-y_test11), 2)  #
R_squared_val = 1-SS_res/SS_tot
print('R^2 val:', R_squared_val)

rmse = np.sqrt(mean_squared_error(y_test11,Y_pred_test11))
print('RMSE:',rmse)
rmse_lin_list.append(rmse)


R2_train_lin_list.append(R_squared_train)
R2_val_lin_list.append(R_squared_val)


R^2 train: 0.878681319362013
R^2 val: 0.8869380322882803
RMSE: 0.015520512608135528


In [129]:
X_train11 = X_train4
y_train11 = y_train4
X_test11 = X_test4
y_test11 = y_test4

w_pre11=np.linalg.pinv(X_train11).dot(y_train11)  #
y1_pred11 = X_train11.dot(w_pre11)                 #
np.power(np.linalg.norm(y1_pred11-y_train11), 2)  #


Y_pred_train11 = X_train11.dot(w_pre11)        #
Y_mean = np.mean(y_train11)                  #
SS_tot = np.sum(np.power(y_train11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_train11-y_train11), 2)   #
R_squared_train = 1-SS_res/SS_tot
print('R^2 train:', R_squared_train)   


Y_pred_test11 = X_test11.dot(w_pre11)     #
Y_mean = np.mean(y_test11)    #
SS_tot = np.sum(np.power(y_test11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_test11-y_test11), 2)  #
R_squared_val = 1-SS_res/SS_tot
print('R^2 val:', R_squared_val)


rmse = np.sqrt(mean_squared_error(y_test11,Y_pred_test11))
print('RMSE:',rmse)
rmse_lin_list.append(rmse)

R2_train_lin_list.append(R_squared_train)
R2_val_lin_list.append(R_squared_val)


R^2 train: 0.8801262085389552
R^2 val: 0.8811495539706472
RMSE: 0.01579022387370486


In [130]:
X_train11 = X_train5
y_train11 = y_train5
X_test11 = X_test5
y_test11 = y_test5

w_pre11=np.linalg.pinv(X_train11).dot(y_train11)  #
y1_pred11 = X_train11.dot(w_pre11)                 #
np.power(np.linalg.norm(y1_pred11-y_train11), 2)  #


Y_pred_train11 = X_train11.dot(w_pre11)        #
Y_mean = np.mean(y_train11)                  #
SS_tot = np.sum(np.power(y_train11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_train11-y_train11), 2)   #
R_squared_train = 1-SS_res/SS_tot
print('R^2 train:', R_squared_train)   


Y_pred_test11 = X_test11.dot(w_pre11)     #
Y_mean = np.mean(y_test11)    #
SS_tot = np.sum(np.power(y_test11 - Y_mean, 2))   #
SS_res = np.power(np.linalg.norm(Y_pred_test11-y_test11), 2)  #
R_squared_val = 1-SS_res/SS_tot
print('R^2 val:', R_squared_val)

rmse = np.sqrt(mean_squared_error(y_test11,Y_pred_test11))
print('RMSE:',rmse)
rmse_lin_list.append(rmse)


R2_train_lin_list.append(R_squared_train)
R2_val_lin_list.append(R_squared_val)


R^2 train: 0.8825319809331221
R^2 val: 0.8710958848215733
RMSE: 0.01579220083162629


In [131]:
print(R2_train_lin_list)
print(R2_val_lin_list)
print(rmse_lin_list)

[0.8792878012535912, 0.8822669543759765, 0.878681319362013, 0.8801262085389552, 0.8825319809331221]
[0.8847739876333096, 0.8723043781349946, 0.8869380322882803, 0.8811495539706472, 0.8710958848215733]
[0.015661128397954457, 0.01619188284715462, 0.015520512608135528, 0.01579022387370486, 0.01579220083162629]


In [132]:
print(np.std(R2_train_lin_list))
print(np.std(R2_val_lin_list))
print(np.std(rmse_lin_list))

0.0015579942558319986
0.006449183119411184
0.00022393703746109516


In [ ]:
X_fold1 = feature_space[0:int(len(feature_space)*.2)]
X_fold2 = feature_space[int(len(feature_space)*.2):int(len(feature_space)*.4)]
X_fold3 = feature_space[int(len(feature_space)*.4):int(len(feature_space)*.6)]
X_fold4 = feature_space[int(len(feature_space)*.6):int(len(feature_space)*.8)]
X_fold5 = feature_space[int(len(feature_space)*.8):]


y_fold1 = y_true[0:int(len(feature_space)*.2)]
y_fold2 = y_true[int(len(feature_space)*.2):int(len(feature_space)*.4)]
y_fold3 = y_true[int(len(feature_space)*.4):int(len(feature_space)*.6)]
y_fold4 = y_true[int(len(feature_space)*.6):int(len(feature_space)*.8)]
y_fold5 = y_true[int(len(feature_space)*.8):]


X_train1 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold4], axis=0)
X_test1 = X_fold5
y_train1 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold4], axis=0)
y_test1 = y_fold5

X_train2 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold5], axis=0)
X_test2= X_fold4
y_train2 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold5], axis=0)
y_test2 = y_fold4

X_train3 = np.concatenate([X_fold1,X_fold2,X_fold4,X_fold5], axis=0)
X_test3 = X_fold3
y_train3 = np.concatenate([y_fold1,y_fold2,y_fold4,y_fold5], axis=0)
y_test3 = y_fold3

X_train4 = np.concatenate([X_fold1,X_fold3,X_fold4,X_fold5], axis=0)
X_test4 = X_fold2
y_train4 = np.concatenate([y_fold1,y_fold3,y_fold4,y_fold5], axis=0)
y_test4 = y_fold2

X_train5 = np.concatenate([X_fold2,X_fold3,X_fold4,X_fold5], axis=0)
X_test5 = X_fold1
y_train5 = np.concatenate([y_fold2,y_fold3,y_fold4,y_fold5], axis=0)
y_test5 = y_fold1
